In [ ]:
# install requirements
#!pip install google-cloud-aiplatform

from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ""
vertexai.init(project="vertax-ai-byungkwonc", location="asia-northeast3")

# vertex ai connection test
model = GenerativeModel(model_name='gemini-pro')
user_message = "인공지능에 대해 한 문장으로 말하세요."
resp = model.generate_content(user_message)
print(resp.text)

# video recognize
model = GenerativeModel("gemini-1.0-pro-vision")
prompt = """
비디오를 보고 다음 질문에 답하세요:
- 비디오에는 무엇이 나와 있나요?
- 비디오에서 어떤 행동이 일어나고 있나요?
- 이 비디오에 가장 적합한 태그 10개를 제시해 주세요.
"""
video = Part.from_uri(
    uri="gs://github-repo/img/gemini/multimodality_usecases_overview/photography.mp4",
    mime_type="video/mp4",
)

contents = [prompt, video]
response = model.generate_content(contents)
print(response.text.strip())

# video 출력
import IPython.display
https_path ="https://storage.googleapis.com/github-repo/img/gemini/multimodality_usecases_overview/photography.mp4"
IPython.display.display(IPython.display.Video(https_path, width=600))
responses = model.generate_content(contents, stream=True)
for response in responses:
    print(response.text.strip(), end="")

In [ ]:
#!pip install pytube, yt_dlp

import yt_dlp
import IPython.display

def download_video_with_ytdlp(youtube_url, filename):
    """yt-dlp를 사용하여 YouTube에서 MP4 비디오를 다운로드하고 파일 이름을 반환합니다."""
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',  # 최고 품질의 비디오와 오디오 형식을 지정
        'outtmpl': f'{filename}.mp4',  # 파일 이름을 지정하고 확장자를 mp4로 설정
        'merge_output_format': 'mp4',  # 비디오와 오디오를 mp4 형식으로 병합
        'ffmpeg_location': '/opt/homebrew/bin/ffmpeg',  # 필요한 경우 FFmpeg 경로 지정
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(youtube_url, download=True)  # YouTube 정보 추출 및 다운로드
        file_name = ydl.prepare_filename(info_dict).replace('.webm', '.mp4')  # 파일 이름을 mp4로 변경
    return file_name  # 파일 이름 반환

# 예시 사용법
youtube_url = 'https://www.youtube.com/watch?v=i-E7NiyRDa0'
#download_video_with_ytdlp(youtube_url, './downloads/video')  # 비디오 다운로드

file_name = "./downloads/video.mp4"
video_path = f"./{file_name}"
IPython.display.display(IPython.display.Video(video_path, width=600,  embed=True))


In [ ]:
import yt_dlp
import os

def download_youtube_content(url, download_type='both', output_dir='downloads'):
    if download_type not in ['video', 'audio', 'both']:
        raise ValueError("download_type must be 'video', 'audio', or 'both'")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    ydl_opts = {
        'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
        'progress_hooks': [lambda d: print(f'다운로드 진행률: {d["_percent_str"]}')],
        'ffmpeg_location': '/opt/homebrew/bin/ffmpeg',  # 필요한 경우 FFmpeg 경로 지정
    }

    if download_type == 'video' or download_type == 'both':
        ydl_opts['format'] = 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best'
        ydl_opts['merge_output_format'] = 'mp4'
    elif download_type == 'audio':
        ydl_opts['format'] = 'bestaudio/best'
        ydl_opts['postprocessors'] = [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }]

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

# 사용 예시
video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"

# 비디오만 다운로드
download_youtube_content(video_url, download_type='video')

# 오디오만 다운로드
download_youtube_content(video_url, download_type='audio')

# 비디오와 오디오 모두 다운로드
download_youtube_content(video_url, download_type='both')